In [7]:
import pyaudio
import struct
import numpy as np 
%matplotlib qt
import matplotlib.pyplot as plt
import time



CHUNK = 1024 * 4
CHANNELS = 1 # Use Micro-phone channel mono sound



#How much audio be processed at a time . 
#How many audio samples per frame we're gonna display 
# so we have 4096 samples per chunk 
FORMAT = pyaudio.paInt16

RATE = 44100 # Sample per second. 44 kHz is most common value


In [9]:
# Audio Waveform Viewer which support windows close process
p = pyaudio.PyAudio()

stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

def plt_handle_close(evt):
    print('Closed Figure!')
    stream.stop_stream()
    stream.close()
    print('stream stopped and close') 
    p.terminate()        
    print('pyaudio terminate()')                 

    
    
fig, ax = plt.subplots() 
x = np.arange(0, 2 *CHUNK, 2)
line, = ax.plot(x, np.random.rand(CHUNK))
ax.set_ylim(0, 255)
ax.set_xlim(0, CHUNK)
plt.setp(ax, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])
fig.canvas.mpl_connect('close_event', plt_handle_close)
print('stream started')

while True:    
    try:
        data = stream.read(CHUNK, exception_on_overflow = False)
        data_int = np.array( struct.unpack(str(2*CHUNK) +'B',data), dtype='b')[::2] + 128 
        line.set_ydata(data_int)
        fig.canvas.draw()
        fig.canvas.flush_events()
    except: 
        stream.stop_stream()
        stream.close()
        print('stream stopped and close') 
        p.terminate()        
        print('pyaudio terminate()')                 
        break



stream started
Closed Figure!
stream stopped and close
pyaudio terminate()
stream stopped and close
pyaudio terminate()


In [ ]:
# Audio Waveform Viewer
# create matplotlib figure and axes
fig, ax = plt.subplots(1, figsize=(15, 7))

# pyaudio class instance
p = pyaudio.PyAudio()

# stream object to get data from microphone
stream = p.open(
    format=FORMAT,
    channels=CHANNELS,
    rate=RATE,
    input=True,
    output=True,
    frames_per_buffer=CHUNK
)

# variable for plotting
x = np.arange(0, 2 * CHUNK, 2)

# create a line object with random data
line, = ax.plot(x, np.random.rand(CHUNK), '-', lw=2)

# basic formatting for the axes
ax.set_title('AUDIO WAVEFORM')
ax.set_xlabel('samples')
ax.set_ylabel('volume')
ax.set_ylim(0, 255)
ax.set_xlim(0, 2 * CHUNK)
plt.setp(ax, xticks=[0, CHUNK, 2 * CHUNK], yticks=[0, 128, 255])

# show the plot
plt.show(block=False)

print('stream started')

# for measuring frame rate
frame_count = 0
start_time = time.time()

while True:
    
    # binary data
    data = stream.read(CHUNK, exception_on_overflow = False)  
    # Set exception_on_overflow = False to avoi
    
    # convert data to integers, make np array, then offset it by 127
    data_int = struct.unpack(str(2 * CHUNK) + 'B', data)
    
    # create np array and offset by 128
    data_np = np.array(data_int, dtype='b')[::2] + 128
    

    
    # update figure canvas
    try:
        line.set_ydata(data_np)
        fig.canvas.draw()
        fig.canvas.flush_events()
        frame_count += 1
        
    except :
        
        # calculate average frame rate
        frame_rate = frame_count / (time.time() - start_time)
        
        print('stream stopped')
        print('average frame rate = {:.0f} FPS'.format(frame_rate))
        break

stream started


In [4]:
stream.stop_stream()
stream.close()
p.terminate()

